# Consolidate Attendance lists

This notebook is for consolidating the attendancelists

In [14]:
# This reload library is just used for developing the REPUBLIC hOCR parser 
# and can be removed once this module is stable.
%reload_ext autoreload
%autoreload 2


#This is needed to add the repo dir to the path so jupyter
# can load the republic modules directly from the notebooks
import os
import sys
import re

# from collections import defaultdict
import pandas as pd
from collections import Counter, defaultdict
# import numpy as np
# #import xmltodict
# import networkx as nx
# from sqlalchemy import text


repo_dir = os.path.split(os.getcwd())[0]

if repo_dir not in sys.path:
    sys.path.append(repo_dir)

# import sys
# !{sys.executable} -m pip install -U jupyter_nbextensions_configurator

In [15]:
# let's repress the warnings, esp. from pandas
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

## Set up the attendancelists for one year

So we have something to work with

In [16]:
import run_attendancelist

In [22]:
runner1730 = run_attendancelist.RunAll(year=1730)


logging to attendancelist.log


In [23]:
%time runner1730.initial_find()

1. finding presidents
288 found
2.find provincial extraordinaris gedeputeerden
CPU times: user 1min 3s, sys: 319 ms, total: 1min 3s
Wall time: 1min 3s


In [24]:
%time runner1730.gather_found_delegates()


3. finding unmarked text
4. joining presidents and delegates
Phrase(, )
Phrase(, )
Phrase(, )
Phrase(, )
Phrase(, )
Phrase(, )
Phrase(, )
Phrase(, )
total 116 found 
CPU times: user 3min 19s, sys: 1.32 s, total: 3min 20s
Wall time: 3min 22s


In [ ]:
%time runner1728.verify_matches()

In [25]:
searchobs = runner1728.searchobs

In [26]:
from collections import Counter, defaultdict
all_marked_names = Counter()
all_marked_presidents = Counter()
vanders = defaultdict(list)
c = 0
for ob in searchobs:
    so = searchobs[ob].matched_text
    for s in so.spans:
        if s.type in ['delegate', 'president']:
            if s.pattern == '':
                pattern = s.pattern
            else:
                pattern = so.item[s.begin:s.end]
            all_marked_names.update([pattern])
            if s.pattern not in all_marked_names:
                if s.pattern not in ['']:
                    if s.pattern.lower() in ['van', 'vander']:
                        vanders[ob].append(s)
            if s.type in ['president']:
                all_marked_presidents.update([pattern])

In [ ]:
runner1730.find_unmarked_text()

In [ ]:
from run_attendancelist import FuzzyKeywordGrouper

In [ ]:
grouped_names = FuzzyKeywordGrouper(list([key for key in all_marked_names.keys() if key != '']))

In [ ]:
from itertools import chain
ll = list(chain(grouped_names.distance_list.keys(), grouped_names.distance_list.values()))

In [ ]:
leftouts = [name for name in all_marked_names.keys() if  name not in ll]

In [ ]:
vtg = grouped_names.vars2graph()

In [ ]:
from numpy import argmax
best_matches = {}
for group in vtg:
    i = argmax([all_marked_names[i] for i in group])
    best_matches[group[i]] = [g for g in group if g != group[i]]
        

In [ ]:
grouped_prezs = FuzzyKeywordGrouper(list([key for key in all_marked_presidents.keys() if key != '']))

In [ ]:
pvtg = grouped_prezs.vars2graph()

In [ ]:
from numpy import argmax
best_match_pres = {}
for group in pvtg:
    i = argmax([all_marked_presidents[i] for i in group])
    best_match_pres[group[i]] = [g for g in group if g != group[i]]
        

In [ ]:
best_match_pres

In [ ]:
from collections import defaultdict
expandables = defaultdict(list)
for k1 in best_match_pres.keys():
    for k2 in best_matches.keys():
        x1, x2 = sorted([k1,k2], key=lambda z: len(z))
        if x1.lower() in x2.lower() and x1!=x2:
            expandables[x1].append(x2)

In [ ]:
#now have to merge them
expandables

In [ ]:
mark_for_delete = []
for i in best_matches.keys():
    if len(i.split(' ')) < 2 and str.islower(i[0]):
            mark_for_delete.append(i)

In [ ]:
mark_for_delete

In [ ]:
def pairs(seq):
    result = []
    for i in enumerate(seq):
        if i[0] < len(seq)-1:
            result.append((i[1], seq[i[0]+1]))
    return result

In [ ]:
reductible = []
for pair in pairs(list(best_matches)):
    if pair[0] not in mark_for_delete:
        npair = sorted(pair, key=lambda x: len(x))
        if npair[0].lower() in npair[1].lower():
            reductible.append(pair)
reductible

In [ ]:
levenst_vals = run_attendancelist.levenst_vals
#runner1730.framed_gtlm.variants.apply(lambda x: levenst_vals(x, phrase))

In [ ]:
variants = []
for k, v in best_matches.items():
    if k not in mark_for_delete:
        if k in expandables:
            print (k)
        # we substitute the key for the longer pattern
            k = expandables[k][0]
        
        phrase = {'phrase': k, 'label':k, 'variants': v} 
        variants.append(phrase)


In [ ]:
import regex

all_phrases = [v['phrase'] for v in variants]
found = runner1730.framed_gtlm
for phrase in all_phrases:
    match = found.variants.apply(lambda x: levenst_vals(x, phrase))
    print(phrase, list(found.loc[match].name)[0])

In [ ]:
vanpat = re.compile('vande[rn]?|([dt]+e?[rn]?)\s+|van', re.I)
for phrase in all_phrases:
    print(phrase, [x.strip() for x in vanpat.split(phrase) if x and x.strip() !=''])

In [ ]:

def is_namepart(variant, found=found):
    result = {}
    result['phrase'] = variant
    result['contained'] = False
    result['exactname'] = False
    match = found.variants.apply(lambda x: levenst_vals(x, phrase))
    matchname = list(found.loc[match].name)[0]
    refid = list(found.loc[match].ref_id)[0]
    result['matchname'] = matchname
    result['id'] = refid
    searchpat = [x.strip() for x in vanpat.split(phrase) if x and x.strip() !='']
    for p in searchpat:
        if p in matchname:
            result['exactname'] = True
            if len(phrase)<len(matchname):
                result['contained'] = True
    return result

In [ ]:
all_phrases = [v['phrase'] for v in variants]
found = runner1728.framed_gtlm
matches = []
for phrase in all_phrases:
    matches.append(is_namepart(phrase))


In [ ]:
groupedmatches = defaultdict(list)
for item in matches:
    groupedmatches[item['id']].append(item)
groupedmatches

In [ ]:
from fuzzy_search.fuzzy_phrase_searcher import FuzzyPhraseSearcher
from fuzzy_search.fuzzy_phrase_model import PhraseModel
fuzzysearch_config = {
    "char_match_threshold": 0.8,
    "ngram_threshold": 0.6,
    "levenshtein_threshold": 0.5,
    "ignorecase": True,
    "ngram_size": 3,
    "skip_size": 1,
}
fs = FuzzyPhraseSearcher(fuzzysearch_config)
model = PhraseModel(variants, config=fuzzysearch_config)
fs.index_phrase_model(phrase_model=model)

In [ ]:
from random import choices
obs = choices(list(searchobs.keys()), k=25)

In [ ]:
name_mappings = run_attendancelist.reverse_dict(best_match_pres)
name_mappings.update(run_attendancelist.reverse_dict(best_matches))

In [ ]:
import regex
for item in name_mappings:
    match = max(fs.find_matches(item, include_variants=True), key=lambda x: x.levenshtein_similarity)
    print(item, match.label, match.offset)
    p = ''.join(('(?:', item, '){e}'))
    peind = regex.compile(p, flags=regex.BESTMATCH)
    r = peind.search(match.label)
    print(r,'====\n')

In [ ]:
for ob in searchobs:
    so = searchobs[ob].matched_text
    for span in enumerate(so.spans):
        s = span[1]
        if s.type in ['president','delegate']:
    #         pres = so.spans[span[0]-1]
    #         try:
    #             post = so.spans[span[0]+1]
    #         except IndexError:
    #             post = None
    #         m = fs.find_matches(s.pattern, include_variants=True)
            pattern = so.item[s.begin:s.end]
            longpat = name_mappings.get(pattern)
#             if longpat and longpat!='':
#                 
#                 pattern, name_mappings[pattern]
            if not longpat:
                try:
                    longpats = max(fs.find_matches(pattern, include_variants=True), key=lambda x: x.levenshtein_similarity)
                    longpat = longpats.label
                except ValueError:
                    pass
            if longpat and longpat!='':
                p = ''.join(('(?:', longpat, '){e}'))
                peind = regex.compile(p, flags=regex.BESTMATCH)
                r = peind.search(so.item)
                begin = longpat.find(pattern)
                so.set_span(span=r.span(),
                           clas='delegate',
                           delegate_name=s.delegate_name, 
                           score = s.delegate_score)
    #             b = s.begin - phrase.offset
    #             e = b + len(s.delegate_name)
    #             mt = so.item[b:e]
    #             sc = score_levenshtein_distance_ratio(s.delegate_name, mt)
    #             overlap = []
    #             if sc > 0.5:
    #                 if b < pres.end:
    #                     overlap.append( pres.pattern)
    #                 if post:
    #                     if e > post.begin:
    #                         overlap.append(post.pattern)
    #                 so.set_span(span=(b,e),
    #                             clas='delegate',
    #                             delegate_name=s.delegate_name, 
    #                             score = s.delegate_score)


In [ ]:
jitems = ';'.join(so.get_unmatched_text())
items = re.split('[;,\.]+', jitems)
items

In [ ]:
found = run_attendancelist.found_delegates
df = run_attendancelist.abbreviated_delegates
i_s = run_attendancelist.iterative_search
noresults = Counter()
for ob in searchobs:
    so = searchobs[ob].matched_text
    jitems = ';'.join(so.get_unmatched_text())
    items = [item.strip() for item in re.split('[;,\.]+', jitems) if item.strip()!='' and len(item.strip())>2]
    for item in items:
        b = e = delegatename = score = None
#        r = fs.find_matches(item, include_variants=True)
        try:
            longpats = max(fs.find_matches(item, include_variants=True), key=lambda x: x.levenshtein_similarity)
            if getattr(longpats, 'string')!='':
                longpat = longpats.string
                b = longpats.offset
                e = b + len(longpats.string)
                delegatename = longpats.label
                score = longpats.levenshtein_similarity
                # print(longpats)
            else:
                match = found.variants.apply(lambda x: levenst_vals(x, item))
                print(item, list(found.loc[match].name)[0])
        except ValueError:
            # try:
            #     match = found.variants.apply(lambda x: levenst_vals(x, item))
            #     b = so.item.find(item)
            #     e = b + len(item)
            #     delegatename = list(found.loc[match].name)[0]
            #     score = 0
            # except IndexError:
            #     try:
            #         match = i_s(name=item, year=1730, debug=False, df=df)
            #         if len(match)>0:
            #             b = so.item.find(item)
            #             e = b + len(item)
            #             delegatename = list(match.name)[0]
            #             score = 0
            #     except IndexError:
                    noresults.update([item])
        # if b:
        #     so.set_span(span=(b,e),
        #     clas='delegate',
        #     delegate_name=delegatename,
        #     score = score)

# Strategy for identifying unknown spans

- run regular checks
- gather unmarked snippets
- group them, with references to days (how?)
- gather most frequent non-identified groups
    - can we identify them?
    - if we can, apply identification to references
    - if not, make unidentified references (which id?) and add it to the list of references
- how many unidentified spans have we still got left
- filter out spurious words (frequency?)
- how many are there still. Do they look like names? (begin with 'van' pat, capitals, are they part of composite names, that we have identified)

further processing:

- do we have double identification that may be reduced, because they are part of names
- after that:
    - are there any outlying presidents: that occur only once or out of place (in a differently identified row)
    - can we do something with the order of delegates

In [ ]:
# let's make an object that points at attendancelists. 
# Note that for use in comparisons that involve typechecking for strings
# we may have to take a roundtrip converting it to a real string and later retrieve the actual object, but that's fine

from collections import UserString
class StringWithContext(UserString):
    def __init__(self, item='', reference=None):
        self.data = item
        self.references = []
        
    def set_reference(self, reference):
        self.references.append(reference)
        
    def get_references(self):
        return self.references



In [ ]:
# test

x = StringWithContext('bla')
type(x)
x.lower()
x.set_reference('q23')
'q23' in x.references




In [ ]:
# gather unmarked snippets
# group them, with references to days by using the stringwithcontext object

found = run_attendancelist.found_delegates
df = run_attendancelist.abbreviated_delegates
i_s = run_attendancelist.iterative_search
noresults = []

for ob in searchobs:
    so = searchobs[ob].matched_text
    jitems = ';'.join(so.get_unmatched_text())
    items = [item.strip() for item in re.split('[;,\.]+', jitems) if item.strip()!='' and len(item.strip())>2]
    for item in items:
        contextitem = StringWithContext(item)
        contextitem.set_reference(so)
        noresults.append(contextitem)
        

In [ ]:
#these are unmarked spans in the attendance lists
#probably we best sort these by hand and try and assign them 
# Counter([i for i in noresults]).most_common()
noresultscounter = Counter(noresults)

In [ ]:
# gather most frequent non-identified groups

from republic.helper.utils import best_match
unmatched_grouped = FuzzyKeywordGrouper([str(k) for k in noresultscounter.keys()]).vars2graph()
consolidated_groups = {}
for group in unmatched_grouped:
    i = argmax([noresultscounter[i] for i in group])
    consolidated_groups[group[i]] = [g for g in group if g != group[i]]
        
consolidated_groups

In [ ]:
xgroups = {}
for g in consolidated_groups:
    ks = [g] + consolidated_groups[g]
    r = list(filter(lambda i: i in ks, noresults))
    xgroups[g] = r

In [ ]:
js = run_attendancelist.junksweeper
suspects = []
for key in xgroups:
    jmatches = js.find_matches(key, include_variants=True, use_word_boundaries=False)
    if len(jmatches)>0:
        bestjmatch = max(jmatches, key=lambda m: m.levenshtein_similarity)
        if not vanpat.search(bestjmatch.string):
            suspects.append(key)
        
    

In [ ]:
suspects

In [ ]:
def match_from_dict(pattern, searcher=fs, founddb=found):
    """pattern should be string in context"""
    
    longpats = fs.find_matches(pattern, include_variants=True)
    if len(longpats)>0:
        result = max(longpats, key=lambda x: x.levenshtein_similarity)
    else:
        match = found.variants.apply(lambda x: levenst_vals(x, item))
        result = list(found.loc[match].name)
        if len(result)==0:
            result = pattern
    return result

In [ ]:
def make_span(keyw):
    for p in xgroups[keyw]:
        obs = p.get_references()
        for ob in obs:
            s = re.search(re.escape(str(p)), ob.item)
            print(ob.item[:s.span()[0]], "##", p, "##", ob.item[s.span()[1]:])
        

In [ ]:
len(match_from_dict('de Raadt'))

In [ ]:
for key in xgroups:
    if key not in suspects:
        r = match_from_dict(key)
        print(key)
        if r:
            make_span(key)

In [ ]:
make_span('van Cattenburgh')

In [ ]:
#     can we identify them?
#     if we can, apply identification to references
#     if not, make unidentified references (which id?) and add it to the list of references

for key in xgroups:
    if key not in suspects:
        # print('\n\n=======\n', key) 
        splt = re.split(vanpat, key)
        if len(splt)>1:
            print(splt[-1])
            print(fs.find_matches(splt[-1], include_variants=True))
        # print(is_namepart(key))
            match = found.variants.apply(lambda x: levenst_vals(x, key))
            print(list(found.loc[match].name))
            if len(match)==0:
                match = i_s(name=item, year=1730, debug=False, df=df)
                if len(match)>0:
                    print(match[0])
    

In [ ]:
found = run_attendancelist.found_delegates
df = run_attendancelist.abbreviated_delegates
i_s = run_attendancelist.iterative_search


In [ ]:
m = searchobs['session-1730-01-03-num-1'].matched_text
m.get_fragments()

In [ ]:
fuzzysearch_config = {
    "char_match_threshold": 0.8,
    "ngram_threshold": 0.6,
    "levenshtein_threshold": 0.5,
    "ignorecase": True,
    "ngram_size": 3,
    "skip_size": 1,
}
import random
from republic.analyser.attendance_lists.pattern_finders import make_province_searcher
ps = make_province_searcher(config=fuzzysearch_config)
ks = random.choices(list(runner1730.searchobs.keys()), k=20)
# pres = []
# prs = []
for T in ks:
    ob = runner1730.searchobs[T].matched_text
    print(T, ob.get_fragments())


In [ ]:
searchobs = runner1730.searchobs
from IPython.display import HTML
htmlout=[]
for T in searchobs:
    ob = searchobs[T].matched_text
    url = searchobs[T].make_url()
    ob.mapcolors()
    rest = ob.serialize()
    rest = f"\n<h4>{T}</h4>\n" + rest
    if url:
        rest += f"""<br/><br/><a href='{url}'>link naar {T}-image</a><br/>"""
    htmlout.append(rest)
#out.reverse()
HTML("<br><hr><br>".join(htmlout))

In [ ]:
for T in ks:
    ob = runner1730.searchobs[T].matched_text
    txt = ob.item
    res = ps.find_matches(text=txt, include_variants=True)
    print (T,txt, [r for r in res if r.label=='heere'])

In [ ]:
from IPython.display import HTML
t_out=[]
searchobs = runner1730.searchobs
for T in searchobs:
    ob = searchobs[T].matched_text
    url = searchobs[T].make_url()
    ob.mapcolors()
    rest = ob.serialize()
    rest = f"""\n<tr><td><strong>{T}</strong></td><td>{rest}</td>"""
#     if url:
#         rest += f"""<td><a href='{url}'>link naar {T}-image</a></td>"""
    rest += "</tr>"
    t_out.append(rest)
#out.reverse()
outtable = "".join(t_out)
#HTML(f"<table>{outtable}</table>")


In [ ]:
year = runner1730.year
with open(f"/Users/rikhoekstra/Downloads/{17}_sample_nocheck.html", 'w') as flout:
    flout.write(f"<html><body><h1>results for {year}</h1>\n")
    flout.write(f"<table>{outtable}</table>")
    flout.write("</body></html>")

In [ ]:
ps = run_attendancelist.make_province_searcher(config=fuzzysearch_config)
i = searchobs[obs[5]].matched_text.item
ps.find_matches(i, include_variants=True)

Most likely the verification code needs an overhaul, but we'll see about that later.

## Experiment: find delegates in resolutions

## remediate text length

Sometimes, there are faulty attendancelists, that contain parts of the following resolutions. Further on, there is more finegrained code, but here we cutdown the size of the texts on basis of the mean text length, as there is not really an attendancelist that should be much longer than other.

## Presidents code

In [ ]:
runner1730.presidents

In [ ]:
searchobs = runner1730.searchobs
met_en_zonder = {}
met_pr = {}
zonder_pr = {}
for T in searchobs:
    ob = searchobs[T].matched_text
    pr = [s for s in ob.spans if s.type == 'president']
    if len(pr)==0:
        zonder_pr[T]=pr
    else:
        met_pr[T]=pr
met_en_zonder['met_pr'] = met_pr
met_en_zonder['zonder_pr'] = zonder_pr
met_en_zonder

Let's calculate some statistics for these presidents:


In [ ]:
import statistics
starts = [v[0].begin for v in met_en_zonder['met_pr'].values()]
ends = [v[0].end for v in met_en_zonder['met_pr'].values()]
lengths = [v[0].end - v[0].begin for v in met_en_zonder['met_pr'].values()]
print("mean beginning and end offsets:")
meanstart = round(statistics.mean(starts),0)
print(meanstart)
meanend = round(statistics.mean(ends),0)
print(meanend)
print("min and max start offsets:")
print("min: ", min(starts),";max: ", max(starts))
print("min and max end offsets:")
print("min: ", min(ends), ";max: ", max(ends))
print("try and find some outlying values")
print("mean: ", round(statistics.mean(lengths), 0))
minlength = min(lengths)
maxlength = max(lengths)
minpres = [v[0].pattern for k,v in met_en_zonder['met_pr'].items() if v[0].end-v[0].begin==minlength ]
print("min: ", minlength , "max: ", maxlength)
#we should do something more with this, but at this point we cannot easily retrieve the spans


We compare try and complete the list of presidents by 1) finding the sessions without president and see if we can find any of them in the first part of the text. This

In [ ]:
dg = DelegateChecker(keywords=keywords, searchobs=runner1730.searchobs)
r = dg.searcher.find_matches('den heere van Isselmuden')[0]
r.phrase
r.levenshtein_similarity

In [ ]:
# this does not generalize much, but we'll leave that for later
from collections import defaultdict
from republic.analyser.attendance_lists.searchers import make_herensearcher
from republic.fuzzy.fuzzy_keyword_searcher import score_levenshtein_distance_ratio
from numpy import argmax


class DelegateChecker(object):
    def __init__(self, keywords, searchobs):
        self.searcher = make_herensearcher(keywords=keywords)
        self.context_searcher = make_herensearcher(keywords={'PRAESENTIBUS':[], 'Den Heere':[]})
        self.searchobs = searchobs
        self.likely_prez = defaultdict(list)
        self.checkdeps = self.make_noprez()
        self.ks = self.make_keys()
        
    def make_keys(self):
        ks = list(self.checkdeps.keys())
        ks.sort() # we depend on the key order
        return ks
        
    def get_vicinity_presidents(self,t):
        ti = self.ks.index(t)
        mn = max(ti-7, 0) 
        mx = min(ti+7, len(self.ks)-1)
        likely_prez = defaultdict(list)
        for pt in self.ks[mn:mx]:
            if self.checkdeps[pt]:
                president = self.checkdeps[pt][0]
                name = president.delegate_name
                pattern = president.pattern
                likely_prez[name].append(pattern)
        return likely_prez
        # for p in likely_prez:
        #     ra.all_matched
#         xsisting_kws = list(self.searcher.keyword_index.keys())
#         kws = [k for k in list(likely_prez.keys()) if k not in xsisting_kws]            
#         self.searcher.index_keywords(kws)
#         for k in kws:
#             for variant in likely_prez[k]:
#                 self.searcher.index_spelling_variant(k, variant=variant)
    
    def check(self):
        results = []
        chosen = None
        for t in self.ks:
            start = 0
            if self.checkdeps[t] is None:
                txt = self.searchobs[t].text
                if len(txt) > 30:
                    candidate = None
                    res = self.searcher.find_matches(text=txt, include_variants=True)
#                     if len(res) == 0:
#                         print('breaking', t)
#                         break
#                     hres = [r for r in res if r.label=='Den Heere']
#                     if hres:
#                         sres = min(hres, key=lambda x: x.offset)
#                         if len(sres) > 0:
#                             start = sres.offset  + len(sres.string)       
#                     pres = [r for r in res if r.label=='PRAESENTIBUS']
#                     if len(pres) > 0:
#                         eres = min(pres, key=lambda x: x._offset)
#                         end = eres.offset
#                     else:
#                         end = start + 20
#                     stxt = txt[start:end]
#                     stxt = stxt.strip()
                    likely_prez = self.get_vicinity_presidents(t)
                    mxs = []
                    for i in res:
                        lmax = []
                        for p in likely_prez:
                            lmax.append(max([score_levenshtein_distance_ratio(i.string, v) for v in likely_prez[p]]))
                        l = argmax(lmax)
                        mxs.append(l)
                    chosen = argmax(mxs)
                    score = mxs[chosen]
#                    print(score, chosen, len(res), t)
                    if score > 0.8:
    #                     for cand in likely_prez:
    #                         candidate = max([(i, score_levenshtein_distance_ratio(stxt, i)) for i in likely_prez[cand]])
                        try:
                            candidate = res[chosen]
                        except IndexError:
                            print(chosen, len(res), t)
#                     else:
#                         prop = self.searcher.find_matches(stxt, include_variants=True)
#                         if len(prop)>0:
#                             candidate = max(prop, key=lambda x: x.levenshtein_similarity)
#                         else:
#                             candidate = None
                    if candidate:
                        b = candidate.offset
                        e = b + len(candidate.string)
                        etxt = txt[:b], txt[b:e], txt[e:]
                    else:
                        etxt = txt
                    results.append([t, etxt]) 
        return results

    
    def make_noprez(self):
        noprez = {}
        for T in self.searchobs:
            ob = self.searchobs[T]
            p = [s for s in ob.get_spans() if s.type=='president']
            if p:
                noprez[T] = p
            else:
                noprez[T] = None
        return noprez

In [ ]:
keywords = {i.name:[v.form for v in i.variants['general']] for i in runner1730.moregentlemen}


In [ ]:
keywrds = {}
for e in runner1730.all_matched:
    i = runner1730.all_matched[e]
    nm = i["name"]
    f = [i["m_kw"]]
    f.extend([v[1] for v in i["variants"]])
    keywrds[nm]=list(set(f))
    

In [ ]:
dlgchk = DelegateChecker(keywords=keywrds, searchobs=runner1730.searchobs)

In [ ]:
searchobs = dlgchk.searchobs

In [ ]:
srch = dlgchk.searcher
zscore = []
for t in dlgchk.ks:
    if dlgchk.checkdeps[t] is None:
        sessie = searchobs[t]
        txt = sessie.text
        res = srch.find_matches(txt, include_variants=True)
        likely_prez = dlgchk.get_vicinity_presidents(t)
        mxs = []
        for i in res:
            lmax = []
            for p in likely_prez:
                lmax.append(max([score_levenshtein_distance_ratio(i.string, v) for v in likely_prez[p]]))
            l = argmax(lmax)
            mxs.append(l)
        if len(mxs) > 0:
            chosen = argmax(mxs)
            score = mxs[chosen]
            omax = min(res, key=lambda x: x.offset)
            zscore.append([t, res[chosen], omax] )

## Try and fill up spaces in the spans

We had to find delegates by searching for their names. But sometimes names that appear in the database ('the canonical name') differs from the one we find in the actual text. Leaving aside OCR errors, this has different forms:

- the actual spelling of the name is different (_ten Brincke_ vs _ten Brinke_; _Isselmude_ vs _Ysselmuden_), but there are no rules as to whether the database or the text spelling is more archaic.
- the name in the text designates the territorial name instead of the family name in the database. For instance: _Noortwijk_ vs _van Aerssen_.
- the name consists of different parts. Because of the matching and tokenization and OCR-errors (especially in separation characters such as . (period) or , (comma)) we could mostly only match one name part. This leaves out many prepositions common in Dutch names (like _van_, _ter_ etc). For example: _Bors_ van Waveren, Taats van _Amerongen_, though in the last case often both parts are matched separately: _Taats_ van _Amerongen_
- the name in the database consists of more or different parts than the name in the text. This may go any way _van Heeckeren_ vs _van Heeckeren van de Brantzenburg_ (this is an especially interesting case, as there are sometimes two _van Heeckeren_ at the same time. Sometimes different parts of the name have been resolved to different delegates, for instance _de Milan Visconti_: _Visconti_ to _de Milan Visconti_ and _Milan_ to _Becker_

Proposed solution:
- try and find the actual occurrence of the names by comparing database and text and by filling out the recognized patterns with surrounding words (that have not been marked or resolve to the same delegate)

In [ ]:
# we first find spans and check for adjacency
searchobs = runner1730.searchobs
allspans = {}
for T in searchobs:
    ob = searchobs[T].matched_text
    allspans[T] = {"afwijkingen":spns2gaps(o), "text":ob.item}

In [ ]:
obs =['session-1730-02-22-num-1',
 'session-1730-09-12-num-1',
 'session-1730-09-06-num-1',
 'session-1730-08-04-num-1',
 'session-1730-10-04-num-1',
 'session-1730-04-04-num-1',
 'session-1730-03-07-num-1',
 'session-1730-11-07-num-1',
 'session-1730-01-27-num-1',
 'session-1730-10-19-num-1',
 'session-1730-09-30-num-1',
 'session-1730-03-27-num-1',
 'session-1730-05-17-num-1',
 'session-1730-11-24-num-1',
 'session-1730-07-21-num-1',
 'session-1730-12-22-num-1',
 'session-1730-07-03-num-1',
 'session-1730-05-27-num-1',
 'session-1730-03-07-num-1',
 'session-1730-02-07-num-1',
 'session-1730-02-22-num-1',
 'session-1730-03-24-num-1',
 'session-1730-11-29-num-1',
 'session-1730-07-08-num-1',
 'session-1730-12-13-num-1']

setup objects for testing
from random import choices
obs = choices(list(allspans.keys()), k=25)


In [ ]:
from collections import defaultdict, Counter
def spns2gaps(ob):
    o = ob.spans
    
    afwijkingen = defaultdict(dict)
    if o[0].end == 0:
        afwijkingen[o[0]]['trailer'] = (o[0].end, o[1].begin)
    for pair in pairs(o):
        s1 = pair[0]
        s2 = pair[1]
        if s1.delegate_id == s2.delegate_id:
            afwijkingen[s1]['merge_propose'] = (s1.begin, s2.end)
        afwijkingen[s1]['trailer'] = (s1.end, s2.begin)
    if o[-1].end<len(ob.item):
        afwijkingen[o[-1]]['trailer'] = (o[-1].end, len(ob.item))
    return afwijkingen

In [ ]:
from fuzzy_search.fuzzy_phrase_searcher import FuzzyPhraseSearcher
fuzzysearch_config = {
    "char_match_threshold": 0.8,
    "ngram_threshold": 0.6,
    "levenshtein_threshold": 0.5,
    "ignorecase": True,
    "ngram_size": 3,
    "skip_size": 1,
}
def make_searcher(config=fuzzysearch_config, nms=[]):
    pr_searcher = FuzzyPhraseSearcher(config)
    phrases = []
#     for nm in nms:
#         phrase = {'phrase':nm,
#                   'label':'',
#                 }
#     basephrase = [{'phrase':"extraordinaris Gedeputeerden uyt de provincie van",
#                   'label':'extraordinaris',
#                   'variants':[]}]
    phrases = nms
    #pmodel = PhraseModel(model=phrases, config=config)
    #print(phrases, phrase_model)
    pr_searcher.index_phrases(phrases)
    return pr_searcher

In [ ]:
FuzzyKeywordGrouper(run_attendancelist.make_groslijst(searchobs)).vars2graph()

In [ ]:
so = searchobs[obs[3]].matched_text
for span in enumerate(so.spans):
    s = span[1]
    if s.type in ['unmarked', 'delegate', 'president']:
        searchwindow = so.item[s.begin-20:s.end+20]
        if s.pattern != '':
            pattern = s.pattern
        else: 
            pattern = f"sw: {searchwindow}"
        print(f"type:{s.type}, pattern: {pattern}, name: {s.delegate_name} \n========\n")


In [ ]:
for umt in so.get_unmatched_text():
    print(fs.find_matches(umt))

In [ ]:
fs.phrase_model

In [ ]:
[d for d in all_marked_presidents if d=='Bout']


In [ ]:
searchobs = runner1730.searchobs
from IPython.display import HTML
htmlout=[]
for T in searchobs:
    ob = searchobs[T].matched_text
    url = searchobs[T].make_url()
    ob.mapcolors()
    rest = ob.serialize()
    rest = f"\n<h4>{T}</h4>\n" + rest
    if url:
        rest += f"""<br/><br/><a href='{url}'>link naar {T}-image</a><br/>"""
    htmlout.append(rest)
#out.reverse()
HTML("<br><hr><br>".join(htmlout))

In [ ]:
from IPython.display import HTML
t_out=[]
for T in obs:
    ob = searchobs[T].matched_text
    url = searchobs[T].make_url()
    ob.mapcolors()
    rest = ob.serialize()
    rest = f"""\n<tr><td><strong>{T}</strong></td><td>{rest}</td>"""
#     if url:
#         rest += f"""<td><a href='{url}'>link naar {T}-image</a></td>"""
    rest += "</tr>"
    t_out.append(rest)
#out.reverse()
outtable = "".join(t_out)
HTML(f"<table>{outtable}</table>")


In [ ]:
with open(f"/Users/rikhoekstra/Downloads/{year}_sample_check.html", 'w') as flout:
    flout.write(f"<html><body><h1>results for {year}</h1>\n")
    flout.write(f"<table>{outtable}</table>")
    flout.write("</body></html>")

### Sample correctness results

- sample of 25 days from 1730 after corrected run

__results__:
- 0,229 totaal fout
- 0,105121294	fout excl vz
- 0,123989218	fout vz
- 0,770889488	goed (ex vz)

__explanation__:
- no text was recognized falsely
- 1 day was parsed wrong, meaning there was no separation between text and attendance list, but still most of the delegates were recognized correctly
- 2 presidents were wrong (only 'vander' recognized, instead of 'vander Waayen' or 'vander Steen'
- no extraordinaris gedelegeerde uyt de provincie etc were wrong, but 'met vier' and 'met vijf' not recognized
- 77 percent were complete and correctly recognized
- in total 23 percent was not completely recognized
- 54 % of the mistakes more  percent were recognized partially correct (some prepositions were left out)
- 46 percent of the mistakes was not recognized, this is 11 percent of all results


In [ ]:
searchobs = runner1730.searchobs
from IPython.display import HTML
htmlout=[]
for T in searchobs:
    ob = searchobs[T].matched_text
    url = searchobs[T].make_url()
    ob.mapcolors()
    rest = ob.serialize()
    rest = f"\n<h4>{T}</h4>\n" + rest
    if url:
        rest += f"""<br/><br/><a href='{url}'>link naar {T}-image</a><br/>"""
    htmlout.append(rest)
#out.reverse()
HTML("<br><hr><br>".join(htmlout))

In [ ]:
gl = run_attendancelist.make_groslijst(searchobs)
#make_groslijst(presentielijsten=self.searchobs)
c = Counter(gl)
tussenkeys = run_attendancelist.FuzzyKeywordGrouper(keyword_list=list(c.keys()))
dralist = tussenkeys.vars2graph()
# unmarked_text = sweep_list(dralist, junksweeper=self.junksweeper)
# return unmarked_text
dralist

In [ ]:
runner1730.find_unmarked_text(sweep=False)

### Comment

This actually works very well. We need to check on the unmarked text to see what is still left out.

## Check delegates on their neighbours.

As explained above, delegates do not appear randomly in in the attendance lists, as these are dictated by the order of the provinces they represent. Because the group of delegates evolves over time and may vary any given day, there is no fixed order of delegates, but still we may assume often delegates have the same neighbours in the attendance list. Therefore we calculate the likely neighbours for delegates from the raw lists of spans. 

In [ ]:
# let's create a sliding window for the spans of a day
from collections import deque

def window(seq, n=5):
    it = iter(seq)
    win = deque((next(it, None) for _ in range(n)), maxlen=n)
    yield win
    append = win.append
    for e in it:
        append(e)
        yield win

### calculate most common last and first delegate

In [ ]:
lspans = Counter()
fspans = Counter()
for T in searchobs:
    ob = searchobs[T].matched_text
    #spns = [(s.delegate_id, s.delegate_name, s.pattern) for s in ob.spans if s.type in ['delegate', 'president'] and s.pattern is not '']
    obspans = [s for s in ob.spans if s.type in ['delegate']]
    if len(obspans)>0:
        lspans.update([obspans[-1].delegate_name])
        fspans.update([obspans[0].delegate_name])

In [ ]:
lspans.most_common()

In [ ]:
fspans.most_common()

In [ ]:
searchobs = runner1730.searchobs
wndws = []
delegates = {}

for T in searchobs:
    ob = searchobs[T].matched_text
    #spns = [(s.delegate_id, s.delegate_name, s.pattern) for s in ob.spans if s.type in ['delegate', 'president'] and s.pattern is not '']
    spns = [s.delegate_name for s in ob.spans if s.type in ['delegate'] and s.pattern is not '']
    for x in window(spns):
        delgates = x 
        d0 = delgates[0]
        d1 = delgates[1] or (0, 'niemand')
        d2 = delgates[2]
        if d1 not in delegates.keys():
            delegates[d1]={'before': Counter(),
                           'after': Counter()}
        delegates[d1]['before'].update([d0])
        delegates[d1]['after'].update([d2])

In [ ]:
xres = []
for d in delegates:
    xitem = delegates[d]
    r = [d[1], xitem['before'].most_common(1)[0][0], xitem['after'].most_common(1)[0][0]]
    xres.append(r)
    #print(f"{d}, common before: {xitem['before'].most_common(1)}, common after: {xitem['after'].most_common(1)}")


In [ ]:
delegates

In [ ]:
for item in lspans.most_common(5):
    print(item[0], delegates.get(item[0]))

In [ ]:
for item in fspans.most_common(5):
    print(item[0], delegates.get(item[0]))

In [ ]:
for x in xres:
    if x[1][0]==None:
        print(x)

In [ ]:
for item in list(list(x) for x in window(spns))[0]:
    print (item)

In [ ]:
len(wndws)

In [ ]:
from collections import defaultdict
z = []
matches = defaultdict(list)
for T in searchobs:
    ob = searchobs[T]
    for s in ob.matched_text.spans:
        if s.type in ['delegate', 'president'] and s.pattern == '':
            proposed_pattern = ob.text[s.begin:s.end]
            spl = re.split(r'[.,-]+', proposed_pattern)
            if len(spl)>1:
                proposed_pattern = spl[0]
            z.append(proposed_pattern)
            matches[proposed_pattern] = ob.text[s.begin:s.end]

In [ ]:
from collections import defaultdict
z = []
matches = defaultdict(list)
for T in searchobs:
    ob = searchobs[T]
    for s in ob.matched_text.spans:
        if s.type in ['delegate', 'president'] and s.pattern == '':
            proposed_pattern = ob.text[s.begin:s.end]
            
            z.append(proposed_pattern)
            

In [ ]:
matches

In [ ]:
results = []
for ob in allspans:
    res = []
    for s in allspans[ob]["spans"]:
        res[s.get('delegate_name')] = s.get(' pattern')
    results.append(res)

In [ ]:
from fuzzy_search.fuzzy_phrase_searcher import FuzzyPhraseSearcher
from fuzzy_search.fuzzy_phrase_model import PhraseModel

# highger matching thresholds for higher quality OCR/HTR (higher precision, recall should be good anyway)
# lower matching thresholds for lower quality OCR/HTR (higher recall, as that's the main problem)
config = {
    "char_match_threshold": 0.8,
    "ngram_threshold": 0.6,
    "levenshtein_threshold": 0.8,
    "ignorecase": False,
    "ngram_size": 2,
    "skip_size": 2,
}

# initialize a new searcher instance with the config
fuzzy_searcher = FuzzyPhraseSearcher(config)

# create a list of domain keywords and phrases
#variants = [{'phrase': k, 'variants': v['variants']} for k, v in runner1715.all_matched.items()]
variants = []
for k, v in runner1715.all_matched.items():
    vrnts = [f[0] for f in v['variants']]
    vrnts.append(f"{v['variants'][0][0]}.DE")
    vrnt = {'phrase': v['name'],'variants':vrnts}
    variants.append(vrnt)
# create a PhraseModel object from the domain phrases
phrase_model = PhraseModel(model=variants,)
#phrase_model.add_phrases(ekwz.keys())
# phrase_model.add_variants(variants)
fuzzy_searcher.index_phrase_model(phrase_model=phrase_model)

In [ ]:
phrase_model

In [ ]:
r = fuzzy_searcher.find_matches(text='Tamminga', include_variants=True,  use_word_boundaries=False)

In [ ]:
result = {}
for i in list(set(z)):
    result[i] = fuzzy_searcher.find_matches(i, include_variants=True, use_word_boundaries=False)

In [ ]:
f

In [ ]:
str(match.variant.exact_string)

In [ ]:
for match in r:
    print(match.levenshtein_similarity, match.character_overlap, match.ngram_overlap, match.variant)
    print(score_levenshtein_distance_ratio(str(match.variant.exact_string),str(match.phrase.exact_string) ), str(match.variant.exact_string),str(match.phrase.exact_string))

In [ ]:
from scipy.special import softmax
from numpy import argmax
def score_match(match):
    result  = sum([match.levenshtein_similarity, match.character_overlap, match.ngram_overlap, score_levenshtein_distance_ratio(str(match.variant.exact_string),str(match.phrase.exact_string))])
    return result

softscore = softmax([score_match(match) for match in r])
i = argmax(softscore)
r[i]

In [ ]:
result

In [ ]:
res = fuzzy_searcher.find_matches(text="Heer. dikke Heeren", include_variants=True, use_word_boundaries=False, allow_overlapping_matches=True)
for item in res:
    print(item.score_levenshtein_similarity())

In [ ]:
matchsearch = FuzzySearcher(config=fuzzysearch_config)
phrase
kws = defaultdict(list)
matcher = {}
for key in all_matched:
    variants = all_matched[key].get('variants')
    keyword = all_matched[key].get('m_kw')
    idnr = all_matched[key].get('id')
    name = all_matched[key].get('name')
    for variant in variants:
        kws[keyword].append(variant[0])
        matcher[variant[0]] = {'kw':keyword, 'id':idnr, 'name':name}
matchsearch.index_keywords(list(kws.keys()))
for k in kws:
    for v in kws[k]:
        matchsearch.index_spelling_variant(keyword=k,variant=v)

In [ ]:
for item in list(set(z)):
    print(runner1715.matchfnd.match_candidates(text=item))

The match and span object takes the unmarked text from the objects and tries to match them with a delegate using the code for identifying delegates. But the failures in the ocr complicates this so that we have to use isolated strings, instead of splitting the text by commas or periods and newlines. This  drops all sorts of prefixes and in the case of a composite name such as _Taats van Amerongen_ it also either makes a name match twice or match part of it with another name. Also the prefixes like _van_ can get matched to the wrong names. So what we will do is match the names and then return to the list of spans and see if we cannot make a better match with the whole names. We would also like  to include previously acquired knowledge for better matching, but we have to think about that more.
Another thing is that we may overfit existing names, but we'll see about that.

In [ ]:
from fuzzy_search.fuzzy_searcher import FuzzySearcher
from fuzzy_search.fuzzy_phrase_model import PhraseModel

config = {'char_match_threshold': 0.7,
 'ngram_threshold': 0.6,
 'levenshtein_threshold': 0.6,
 'ignorecase': False,
 'ngram_size': 2,
 'skip_size': 2}

sps = FuzzySearcher(config)

spns = [(s.pattern, s.get_delegate()) for s in ob.spans if s]
kws = []
for s in spns:
    nm = s[1]['name']
    d_id = s
#    if nm == 'Taets van Amerongen':

phrase_model = PhraseModel(model=variants, )
phrase_model.add_phrases(kws)
sps.index_phrase_model(phrase_model=phrase_model)

In [ ]:
spns

In [ ]:
matchsearch = FuzzyPhraseSearcher(config=fuzzysearch_config)
kws = defaultdict(list)
matcher = {}
for key in all_matched:
    variants = all_matched[key].get('variants')
    keyword = all_matched[key].get('m_kw')
    idnr = all_matched[key].get('id')
    name = all_matched[key].get('name')
    for variant in variants:
        kws[keyword].append(variant[0])
        matcher[variant[0]] = {'kw':keyword, 'id':idnr, 'name':name}
matchsearch.index_keywords(list(kws.keys()))
for k in kws:
    for v in kws[k]:
        matchsearch.index_spelling_variant(keyword=k,variant=v)

In [ ]:
deputyregister = defaultdict(list)
search_results = {}
fm = FndMatch(year=1728,
                 #patterns=pats,
                 register=register,
                 rev_graph=transposed_graph,
                 searcher=herensearcher,
                 junksearcher=junksweeper,
                 df=abbreviated_delegates)
for T in searchobs:
    ob = searchobs[T].matched_text
    mo = MatchAndSpan(ob, junksweeper=junksweeper, previously_matched=previously_matched, match_search=matchsearch)
    for span in ob.spans:
        if span.type in ['president', 'delegate']:
            if span.pattern == '':
                span.set_pattern(pattern=ob.item[span.begin:span.end])
            r = fm.match_candidates(heer=span.pattern)
            span.set_delegate(delegate_id=r.id, 
                              delegate_name=r.proposed_delegate, 
                              delegate_score=r.score)

In [ ]:
matchfnd

## Continue matching

In [ ]:
defo = defaultdict(list)
for item in ob.get_unmatched_text():
    item = [i.strip() for i in re.split('[\.|,]', item)]
    for i in item:
        identified = ''
        res = {}
        r2 = {}
        for g in moregentlemen:
            mn = 0
            co = 0
            mc = 0
            vrts = g.variants['general']
            dr = [score_levenshtein_distance_ratio(x.form, i) for x in vrts if x.heerid==g.heerid]
            if len(dr)>0:
                mn = mean(dr)
            res[g] = {'mn':mn}
            if len(i)<5:
                co = [score_ngram_overlap_ratio(x.form, i, 1) for x in vrts if x.heerid==g.heerid]
                if co and len(co)>0:
                  mc = mean(co)
            res[g]['mc'] = mc
        best_l = max(res, key=lambda key: res[key]['mn'])
        best_c = max(res, key=lambda key: res[key]['mc'])
        if res[best_l]['mn'] > 0.5:
            identified = res[best_l]['mn']
            if best_c != 0:
                if res[best_c] != identified:
                    identified = res[best_c]
        if identified != '':
            delegate = fm.match_candidates(heer=best_l.name)
            identified = delegate
        else: 
            identified = None
        if identified:
            
            name = delegate.proposed_delegate
            m_id = delegate.id
        else:
            name = ''
            m_id = 0
        start = ob.item.find(i)
        end = start + len(i)
        ob.set_span(span=(start, end),pattern=i, clas='deputy', delegate_name=name,
                    delegate_id=m_id)
#         defo[i]={'mn':(best_l,res[best_l]['mn']),
#                  'co':(best_c,res[best_c]['mc'])}
# defo

In [ ]:
def get_mn(k, defo=defo):
    defo = dict(defo)
    v = defo[k]['mn'][1]
    v = float(v)
    return v
    
#         mn = dict(defo[k].get('mn').value
for k in defo:
    v = get_mn(k)
    if v < 0.5:
        res = iterative_search(name=k, year=1728, df=abbreviated_delegates)
        res

In [ ]:
s = moregentlemen[8]
print(s.variants)
for v in s.variants:
    print(score_levenshtein_distance_ratio('an Schwartzenberg', s.name))
    #list(map(lambda x: score_levenshtein_distance_ratio('an Schwartzenberg', x), s.variants))

In [ ]:
v = s.variants['general'][0]
v.form

In [ ]:
res

In [ ]:
span.pattern

In [ ]:
# span.set_delegate(delegate_id=10, 
#                               delegate_name='blop', 
#                               delegate_score=1.0)
span.delegate_id

In [ ]:
all_matched.keys()

In [ ]:
previously_matched

### Some Evaluations (fwiw)

In [ ]:
pd.DataFrame(previously_matched.aggregate("mean")[['levenshtein_distance', 'score']])

In [ ]:
previously_matched.score.value_counts()

In [ ]:
previously_matched.levenshtein_distance.round(2).value_counts().sort_values('index', ascending=False)

In [ ]:
for s in ob.spans:
    print(s, s.idnr, ob.item[s.begin:s.end], s.get_delegate())

## a better searcher and identifier.

The problem: 
We have two ways of identifying name patterns now.

- the first uses pandas based method to identify strings against a database of known delegates and then pulls in some of the external context. This works pretty well for initial identification, but not all names are recognized very well. There is especially a problem by names that have been badly garbled by the text recognition, which is now the case with the OCR, but it will probably not get better if we start using HTR and look up entities in the text. Another issue is that this method has no memory of earlier work, both in the sense that it will not know if earlier matches were found or earlier proposals were marked as false. 
- the fuzzysearcher is very good for spotting name patterns in text as it includes approximate text matching and a number of variants. But it knows nothing about identification and there is no easy way of associating patterns with entities.
- fuzzysearcher is also able to cluster alike word patterns and recluster heterogeneous lists. However, this does not per se bring us closer to identification
- the only way to include previously selected variants is in the fuzzy searcher. Which is useful, but the main problem remains associating this with external knowledge.

This leads to the following requirements:
- a way of associating patterns with identities. But how. Earlier I put the result of a Fuzzysearch pattern clustering in a dataframe, but this included heterogeneous patterns. One possibility is to revise candidates and recategorize weak candidates. The goals is to incrementally improve the patterns associated with a name. This is especially useful if we reuse the objects over many years.
- another idea is to check for internal consistency of the results and try and find inconsistent results. This is especially useful for the presidents, as their number is even more limited than the delegates. The problem is overfitting in which unidentified names get associated with identified names and also new or infrequent delegates may drop out. SO there is a need for manual check.
- another tool may be to make a specialized confusion matrix for delegates

We start with this latest
no we don't. Most of this is already in place. The main problem is that we cannot access the it from 'both' sides, so that we have to rematch all over again. Which is what we do not want, because it slows down the process *and* it introduces mistakes all over again. What we need is memory:
- of previous matches
- of previous associations
- of previous wrong associations
and perhaps more

    

In [ ]:
def get_delegates_from_spans(ob):
    result = {}
    for s in ob.spans:
        if s.type in ['delegate', 'president']:
            result['delegate'] = s.get_delegate()['name']
            result['pattern'] = s.pattern
            result['type'] = s.type
    return result

In [ ]:
all_delegates = [get_delegates_from_spans(searchobs[ob].matched_text) for ob in searchobs]

In [ ]:
from collections import defaultdict
reduced_delegates = defaultdict(Counter)
for item in all_delegates:
    reduced_delegates[item.get('delegate')].update([item.get('pattern')])
    

In [ ]:
reduced_delegates

In [ ]:
moregentlemen = runner1730.moregentlemen
from statistics import mean
from fuzzy_search.fuzzy_string import score_ngram_overlap_ratio
from run_attendancelist import iterative_search, abbreviated_delegates, found_delegates, MatchHeer
day = year = 1730

In [ ]:
unclassified = []
nwheren = []
clean_gentlemen = []
for g in moregentlemen:
    matches = defaultdict(list)
    matchkw = g.matchkw
    matchkw = matchkw.strip()
    nwmatches = {}
    if matchkw == '':
        unclassified = g.variants['general'] # we go to the next instance and look into this later
        continue
    for v in g.variants['general']:
        matches[v.match].append(v)
        for key in matches.keys():
            name = g.name
            nwmatches[matchkw] = []
            s = score_levenshtein_distance_ratio(key, matchkw)
            if s > 0.7:
                nwmatches[matchkw].extend(matches[key])
            else:
                if not re.search('\w+', key):
                    #this is an empty key
                    meanscore = mean([score_ngram_overlap_ratio(kw.form, matchkw, 1) for kw in matches[key]])
                    if meanscore > 0.8:
                         nwmatches[matchkw].extend(matches[key])
        #            print('meanscore: ', key, meanscore)

                # is there no strange 'Van' variant in this?
                elif len(key)<7:
                    s2 = max([score_levenshtein_distance_ratio(key.lower(), van) for van in ['van','vander']])
                    if s2 > 0.8:
                        meanscore = mean([score_levenshtein_distance_ratio(kw.form, matchkw) for kw in matches[key]])
                        nwmatches[matchkw].extend(matches[key])
                        #print('meanscore: ', key, meanscore)
                    else: 
                        unclassified.extend(matches[key])
                else:
                    meanheerscore = mean([score_levenshtein_distance_ratio(kw.form, matchkw) for kw in matches[key]])
                    meankeyscore = mean([score_levenshtein_distance_ratio(kw.form, key) for kw in matches[key]])
                    meanheergramscore = mean([score_ngram_overlap_ratio(kw.form, matchkw, 2) for kw in matches[key]])
                    meankeygramscore = mean([score_ngram_overlap_ratio(kw.form, key, 2) for kw in matches[key]])
                    if meankeyscore>meanheerscore and meankeygramscore>meanheergramscore:
                        #print(key, meankeyscore-meanheerscore, meankeygramscore-meanheergramscore)
                        nwres = iterative_search(key, year=day, df=abbreviated_delegates)
                        rec = {'id': nwres.id.iat[0],
                               'm_kw': key,
                               'name': nwres.name.iat[0],
                               'variants': [(v.form, v.match, v.score) for v in matches[key]]}
                        nwheer = MatchHeer(rec=rec)
                        nwheren.append(nwheer)
    g.variants['new']=nwmatches
        
                
            

In [ ]:
for i in moregentlemen:
    vrs = i.variants['general']
    n = i.name
    m = i.matchkw
    print(n, list(filter(lambda x: score_levenshtein_distance_ratio(x,n)>0.7, vrs)))
    print(m, list(filter(lambda x: score_levenshtein_distance_ratio(x,m)>0.7, vrs)))

In [ ]:
from collections import Counter
allvariants = Counter()
for g in runner1730.moregentlemen:
    allvariants.update(g.variants['general'])
allvariants.most_common(10)

In [ ]:
for i in runner1730.moregentlemen:
    print(i, i.variants)

ok, so we need something do discriminated them. Perhaps the time of their actual delegate appointments?
another idea is to try and match names more closely. But we do not implement that here (as I do not think there is much to gain here as yet)

# Consistency Checks
## Not finished

In [ ]:
unmt = [searchobs[so].matched_text.get_unmatched_text() for so in searchobs]

In [ ]:
wrds = [k[0].form for k in allvariants.most_common() if k[0] not in ['',None]]
fks.find_close_distance_keywords(wrds)


In [ ]:
for nwh in nwheren:
    if nwh.heerid not in all_matched.keys():
        print(nwh.heerid, nwh.name)

In [ ]:
def joinheer(h1, h2):
    if not h1.heerid == h2.heerid:
        raise ValueError('ids not equal')
    forms1 = set([v.form for form in h1.variants])
    forms2 = set([v.form for form in h2.variants])
    difference = forms2.difference(forms1)
    
    for form in difference:
        v1 = [v for v in h2.variants['general'] if v.form==form][0]
        h1.variants['general'].append(v1)
    return h1

In [ ]:
nnwheren = []


In [ ]:
idvals = set([n.heerid for n in nwheren])
clusterednwh = defaultdict(list)
for heerid in idvals:
    clusterednwh[heerid] = [nwh for nwh in nwheren if nwh.heerid == heerid]
    

In [ ]:
from itertools import groupby
ds = {}
sh = sorted(nwheren, key=lambda x: x.heerid)
for k, g in groupby(sh, key=lambda x: x.heerid):
    ds[k] = list(g)
for k in ds:
    h1 = ds[k][0]
    for h2 in ds[k][1:]:
        h1 = joinheer(h1,h2)
    nnwheren.append(h1)

In [ ]:
nnwheren[2].variants

In [ ]:
unclassified

In [ ]:
unob = unclassified[0]
unob.heer


In [ ]:
c = Counter(unclassified)
c.most_common()

In [ ]:
unclas = fks.find_close_distance_keywords([v.form for v in unclassified])
G_unclas = nx.Graph()
d_nodes = sorted(unclas)
for node in d_nodes:
    attached_nodes = unclas[node]
    G_unclas.add_node(node)
    for nod in attached_nodes:
        G_unclas.add_edge(node, nod)
connected_unclas = list(nx.connected_components(G_unclas))


In [ ]:
connected_unclas

In [ ]:
iterative_search('Ten Brincke', year=1728, df=abbreviated_delegates)


In [ ]:
iterative_search('Eck', year=1728, df=abbreviated_delegates)

In [ ]:
for x in connected_unclas:
    for i in x:
        if not re.search('[0-9]+',i):
            print (i)
#         ir = [v for v in unclassified if v.form == i]
#         for vir in ir:
#             print (vir.form, vir.heerid, '\n\n')
            result = herensearcher.find_candidates(i)
            if len(result) > 0:
                eindresult = max(result, key=lambda keyword: keyword['levenshtein_distance'])
                

# Placeholder

In [ ]:
def span_checker(ob):
    """prints all spans from a searchobject"""
    spans = ob.matched_text.spans
    for span in spans:
        txt = ob.matched_text.item[span.begin:span.end]
        span.set_pattern(txt)

        #kand = matchsearch.find_candidates(txt)[0]
        msk = framed_gtlm.vs.apply(lambda x: levenst_vals(x, txt))
        mres = framed_gtlm.loc[msk==msk.max()]
        if len(mres)>0:

    #         setattr(span, 'delegate_id', mres.uuid.iat[0])
    #         setattr(span, 'delegate_name', mres.name.iat[0])
        #print(kand, all_matched[kand])
            span.set_delegate(delegate_id=mres.ref_id.iat[0], delegate_name=mres.name.iat[0], delegate_score=msk.max())
        print(span.type, span.idnr, txt, span.delegate_id, span.delegate_name, span.delegate_score)


In [ ]:
from republic.helper.similarity_match import score_levenshtein_distance
twijfelresults = []
for im in runner1730.all_matched:
    item = runner1730.all_matched[im]
    if score_levenshtein_distance(item['m_kw'], item['name'])<0.6:
        twijfelresults.append(im)
twijfelresults

In [ ]:
len(runner1730.all_matched) - len(twijfelresults)

In [ ]:
runner1730.all_matched[17083]

In [ ]:
def match2pattern(match:dict, bestpattern='', threshold=0.9, year=0):
    goodresults = []
    quarantine = []
    variants = match['variants']
    if bestpattern == '':
            try:
                bestpattern = max(variants, key=lambda x: x[2])[0]
            except TypeError:
                print (variant)
                raise
    if match['score']>threshold:
        results = goodresults
    else:
        results = quarantine
    for variant in variants:
        if variant[2]>threshold:
            result = {}
            result['variant_pattern'] = variant[0]
            result['variant_score'] = variant[2]
            result['variant_keyword'] = bestpattern
            result['match_keyword']=match['m_kw']
            result['match_name'] = match['name']
            result['match_score'] = match['score']
            result['match_id'] = match['id']
            results.append(result)
    return goodresults, quarantine


In [ ]:
year = 1730
indexkws = []
qindexkws = []
for item in runner1730.all_matched:
    matches = runner1730.all_matched[item]
    i_kws, q_kws = match2pattern(matches, year=year)
    indexkws.extend(i_kws)
    qindexkws.extend(q_kws)

In [ ]:
from collections import defaultdict
goodvariants = {}
joined_good_variants = defaultdict(list)
unique_ids = list(set([i.get('match_id') for i in indexkws]))

for k in unique_ids:
    dd = defaultdict(list)
    ikws = [d for d in indexkws if d.get('match_id') == k]
    for d in ikws: # you can list as many input dicts as you want here
        for key, value in d.items():
            if key != 'match_id':
                dd[key].append(value)
    for kie, v in dd.items():
        v = list(set(v))
        if len(v) == 1:
            v = v[0]
        dd[kie] = v
    goodvariants[k] = dd

In [ ]:
goodvariants

In [ ]:
proposed_good_matches = []
for item in qindexkws:
    if item['variant_score'] > item['match_score']:
        item['match_keyword'] = item['variant_pattern']
        item['match_score'] = item['variant_score']
        proposed_good_matches.append(item)
        qindexkws.pop(qindexkws.index(item)) 
        

In [ ]:
vrs = [i['variant_pattern'] for i in indexkws]
vids = [i['match_id'] for i in indexkws]
prids = [i['match_id'] for i in proposed_good_matches]
vids.extend(prids)

In [ ]:
proposed_good_matches

In [ ]:
for item in qindexkws:
    if item['match_id'] in vids:
        print(item)

In [ ]:
len(qindexkws)

In [ ]:
len(proposed_good_matches)

In [ ]:
from statistics import mean
mean([i.get('match_score') for i in indexkws])

In [ ]:
min([i['match_score'] for i in indexkws])

In [ ]:
from republic.analyser.attendance_lists.identify import iterative_search
reslt = {}
for i in twijfelresults:
    item = runner1730.all_matched[i]
    l = 0
    v = ''
    v = max(item['variants'], key=lambda z: len(z[0]))[0]
    reslt[v] = iterative_search(name=v, year=1730, debug=False, df=runner1730.abbreviated_delegates)

In [ ]:
from IPython.core.display import display, HTML
for i in reslt:
    display(i, reslt[i])

G_heren = nx.Graph()
d_nodes = sorted(cl_heren)
for node in d_nodes:
    attached_nodes = cl_heren[node]
    G_heren.add_node(node)
    for nod in attached_nodes:
        G_heren.add_edge(node, nod)
pm_heren = list(nx.connected_components(G_heren))
pm_heren

In [ ]:
item

In [ ]:
from republic.fuzzy.fuzzy_keyword_searcher import FuzzyPhraseSearcher
fuzzysearch_config = {
    "char_match_threshold": 0.8,
    "ngram_threshold": 0.6,
    "levenshtein_threshold": 0.5,
    "ignorecase": False,
    "ngram_size": 2,
    "skip_size": 2,
}
fks = FuzzyPhraseSearcher(config=fuzzysearch_config)
fks.use_word_boundaries = False
kws = [goodvariants[p]['match_keyword'] for p in goodvariants]
false_kws = ['PRAESIDE','PRAESENTIBUS', 'Nihil Actum']
kws.extend(false_kws)
fks.index_keywords(kws)
for kw in kws:
    if kw not in false_kws:
        z = [goodvariants[d] for d in goodvariants if goodvariants[d]['match_keyword'] == kw][0]
        vrs = z["variant_pattern"]
        for v in vrs:
            fks.index_spelling_variant(kw, v)

In [ ]:
suspect_obs = defaultdict(list)
for ob in runner1730.searchobs:
    fragments = runner1730.searchobs[ob].get_fragments()
    fragments.reverse()
    for item in fragments:
        lastdelegate = ''
        ld = item.get('delegate_name')
        if ld in common_last_delegates:
            break
        elif ld == '': 
            p = item.get('pattern')
            r = fks.find_candidates(text=p)
            if len(r)>0:
                r = max(r, key=lambda x: x['levenshtein_distance'])
                if r["match_keyword"] in false_kws:
                    suspect_obs[r["match_keyword"]].append(ob)
                elif r["match_keyword"] in commonlastgoodvariantsvs:
                    begin = item['begin']
                    changeob = runner.searchobs[ob]
                    begin = res[-2]
                    for span in changeob.matched_text.spans:
                        if span.begin>=begin:
                            changeob.matched_text.spans.remove(span)

In [ ]:
commonlastgoodvariants = {v:goodvariants[v] for v in goodvariants if goodvariants[v]['match_name'] in common_last_delegates}

In [ ]:
commonlastgoodvariantsvs = []
for v in goodvariants:
    if goodvariants[v]['match_name'] in common_last_delegates:
        commonlastgoodvariantsvs.extend(goodvariants[v]['variant_pattern'])
commonlastgoodvariantsvs

In [ ]:
lastspans = [f[-1].to_json() for f in out]

In [ ]:
lastfragments[-100]a

In [ ]:
spans = []
for T in ra.searchobs:
    ob = ra.searchobs[T]
    spans.append(ob.get_spans())

In [ ]:
noprez = {}
for T in runner1715.searchobs:
    ob = runner1715.searchobs[T]
    p = [s for s in ob.get_spans() if s.type=='president']
    if p:
        noprez[T] = p
    else:
        noprez[T] = None


In [ ]:
upr = list(set(runner1715.presidents))
upr

In [ ]:
ks = list(noprez.keys())
ks.sort()


In [ ]:
noprez[t][0]

In [ ]:
fks.keyword_index.keys()

In [ ]:
len("'Van Lynden, van Singendonck, IS ter Vergaderinge gelesen de Requeste van Johan Coenraadts Hendricks")

In [ ]:
score_levenshtein_distance_ratio('Van Schwartzenbergh', 'Van Schwart zenbergh')

In [ ]:

fks = make_herensearcher(keywords={i.name:[v.form for v in i.variants['general']] for i in ra.moregentlemen})

In [ ]:
make_herensearcher(keywords={})

In [ ]:
for t in ks:
    if not noprez[t]:
        likely_prez = get_vicinity_presidents(noprez, t, )
        txt = ra.searchobs[t].text
        if len(txt) > 10:
            e = txt.find('PRAESENTIBUS')
            if e > -1:
                stxt = txt[:e]
            else: 
                stxt = txt[:100]
            print(likely_prez.find_candidates(stxt), stxt)
                
         

In [ ]:
run_attendancelist.get_delegates_from_spans(searchobs[T].matched_text)

In [ ]:
for k in ra.searchobs:
    so = ra.searchobs[k]
    sspans = so.get_spans()
    for s in sspans:
        if s.pattern in s.delegate_name:
            if s.delegate_name > s.pattern
            so.get_txt

In [ ]:
ra.all_matched

# OUTPUT

In [ ]:
searchobs = runner1715.searchobs

In [ ]:
from IPython.display import HTML
htmlout=[]
for T in searchobs:
    ob = searchobs[T].matched_text
    url = searchobs[T].make_url()
    ob.mapcolors()
    rest = ob.serialize()
    rest = f"\n<h4>{T}</h4>\n" + rest
    if url:
        rest += f"""<br/><br/><a href='{url}'>link naar {T}-image</a><br/>"""
    htmlout.append(rest)
#out.reverse()
HTML("<br><hr><br>".join(htmlout))


In [ ]:

for T in searchobs:
            print(searchobs[T].resumptionstart, searchobs[T].text)
